<a href="https://colab.research.google.com/github/KOFIYEB/Chatbox_Zephyr7B/blob/main/chatbox_Zephyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbox with Zephyr 7B**

---



We will develop a chatbot using **Zephyr 7B**, a well-regarded conversational model that has been enhanced through fine-tuning with Mistral 7B. Crafted by the Hugging Face H4 team, Zephyr 7B Beta has demonstrated superior performance compared to GPT-3.5, LLama 2 70B, and other models in the 7B size category, as evidenced by the Alpaca Leaderboard rankings. Additionally, Zephyr 7B offers the advantage of being significantly more compact, requiring only 4% of the storage space needed for GPT-3.5.

# **Load Packages**

In [ ]:
# You only need to run this once, even if you re-start the kernel or machine
# Remove the -q to print install output
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets

# **Load Model**

In this section, we initialize the model. You're welcome to substitute it with any conversational or instructional model from Hugging Face, ensuring that you adjust the "roles" accordingly as outlined below. However, this model is for now highly recommended.

In [ ]:
models_dict = {
    "Zephyr 7B Alpha": "HuggingFaceH4/zephyr-7b-alpha",
    "Zephyr 7B Beta": "HuggingFaceH4/zephyr-7b-beta",
    # You can add more chat/instruct models here
}

model_id = models_dict["Zephyr 7B Beta"]

In [ ]:
import torch
from transformers import pipeline

# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

# **Define the role**

specify the role you want the chatbot to adopt. For this example, I'll configure it to function as a sports analyst.

In [ ]:
job_description = "You are a sport analyst expert. Please answer my questions."

Here we execute the model! To conclude the question-answering session, type "stop" or "exit". Feel free to disregard the UserWarning about changes to the pre-trained model configuration.

In [ ]:
messages = [
    {
        "role": "system",
        "content": job_description,
    },
]

exit_terms = ["stop", "exit"]

while True:
    question = input("\nQuestion:")
    if question.lower() in exit_terms:
        break
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    response_start = output.rfind('<|assistant|>')
    print(output[response_start + len('<|assistant|>'):])

Question:Rank the top 10 basketball players of all time

1. Michael Jordan: Widely regarded as the greatest basketball player of all time, Jordan won six NBA championships and was a 14-time NBA All-Star.

2. LeBron James: Four-time NBA champion, four-time NBA MVP, and a 17-time NBA All-Star, James is considered one of the most dominant players in NBA history.

3. Kareem Abdul-Jabbar: Six-time NBA champion and six-time NBA MVP, Abdul-Jabbar holds the all-time NBA scoring record with 38,387 points.

4. Bill Russell: Eleven-time NBA champion and five-time NBA MVP, Russell is the most successful player in NBA history in terms of championships.

5. Magic Johnson: Three-time NBA champion and three-time NBA MVP, Johnson was a dominant point guard in the 1980s.

6. Wilt Chamberlain: Four-time NBA champion and one-time NBA MVP, Chamberlain holds numerous NBA records, including most points in a single game (100).

7. Larry Bird: Three-time NBA champion and three-time NBA MVP, Bird was a master of the mid-range jumper and led the Boston Celtics to an NBA-record 60 wins in the 1985-86 season.

8. Oscar Robertson: One-time NBA champion and one-time NBA MVP, Robertson was the first player to average a triple-double for an entire season.

9. Jerry West: Never won an NBA championship, but West was a 14-time NBA All-Star and is often cited as one of the greatest players never to win a title.

10. Tim Duncan: Five-time NBA champion and two-time NBA MVP, Duncan was a dominant power forward and center for the San Antonio Spurs.

Note: Rankings are subjective and may vary based on individual preferences and opinions.

Question: What does Lebron James have to do to surpass Micheal Jordan as #1

Lebron James has already achieved some of the same accolades as Michael Jordan, such as four NBA championships and four NBA MVP awards. However, to surpass Jordan as the greatest basketball player of all time, James would need to accomplish some of the following:

1. Win more NBA championships: Jordan won six NBA championships in his career, and while James has four, he would need to win two more to surpass Jordan's total.

2. Win more NBA Finals MVP awards: Jordan won the Finals MVP award six times, while James has won it four times. James would need to win another two Finals MVP awards to tie Jordan's total.

3. Achieve better statistical marks: While James has already passed Jordan in some statistical categories, such as career points and assists, there are still some marks that Jordan holds that James has yet to surpass. For example, Jordan holds the NBA record for career playoff points, and James would need to break that record to surpass Jordan in that category.

4. Achieve greater longevity: Jordan played in the NBA for 15 seasons, while James has already played in 18 seasons. If James can continue playing at a high level and win more championships in the later stages of his career, it could help him surpass Jordan in terms of overall impact on the league.

Overall, surpassing Jordan as the greatest basketball player of all time is a tall order for any player, as Jordan's legacy is deeply ingrained in the sport's history. However, James has already achieved an impressive career and still has several years left to make a case for himself as the greatest.

Question: Statistically Lebron has done more than Jordan and is still playing. Why didn't you consider stats like career points, assist point and number of finals reached?

I did consider those statistical categories in my previous answer, and I mentioned that while Lebron James has already passed Michael Jordan in some statistical categories, such as career points and assists, there are still some marks that Jordan holds that James has yet to surpass, like the NBA record for career playoff points.

Regarding the number of Finals reached, both Jordan and James have appeared in the NBA Finals multiple times. Jordan appeared in the Finals a total of 6 times and won 6 championships, while James has appeared in the Finals a total of 9 times and won 4 championships. While James has reached the Finals more times than Jordan, Jordan's winning percentage in the Finals (100%) is higher than James' (67%).

Overall, I believe that while statistics are certainly an important factor to consider when evaluating a player's career, there are other factors that need to be taken into account as well, such as the player's impact on the game, their leadership abilities, and their overall contribution to the team's success. That's why I mentioned a few other factors in my previous answer as well.

Question:stop

**It worked absolutely fine, you can also create your version of ChatGPT 😲**